In [1]:
from xyz2atoms import Molecule, torsion_angle, to_plotly_figure
import numpy as np

## Calculating helical parameters of nucleotide chains from bond length, bond angles and internal-rotation angles
**Shimanouchi T., Mizushima S. On the Helical Configuration of a Polymer Chain // J. Chem. Phys. 23, p. 707-712 (1955)**

$$
A_{ij}^{\varphi} = \begin{bmatrix} 
1 & 0 & 0 \\
0 & \cos\varphi_{ij} & -\sin\varphi_ij \\
0 & \sin\varphi_{ij} & \cos\varphi_{ij}
\end{bmatrix}
$$
$$
A_{i}^{\alpha} = \begin{bmatrix} 
-\cos\alpha_i & -\sin\alpha_i & 0 \\
\sin\alpha_i & -\cos\alpha_i & 0 \\
0 & 0 & 1
\end{bmatrix}
$$
$$
B_{ij} = \begin{bmatrix} 
b_{ij} \\
0 \\
0
\end{bmatrix}
$$

$$
A = \begin{bmatrix}
a_{11} & a_{12} & a_{13} \\ 
a_{21} & a_{22} & a_{23} \\ 
a_{31} & a_{32} & a_{33}
\end{bmatrix} = A_{1}^{\alpha}A_{12}^{\varphi}A_{2}^{\alpha}A_{23}^{\varphi} ... A_{p}^{\alpha}A_{p-1 p}^{\varphi}
$$
$$
B = \begin{bmatrix}
b_1 \\
b_2 \\
b_3
\end{bmatrix} = B_{12} + A_{12}^{\varphi}A_{1}^{\alpha}B_{23} + ... + A_{21}^{\varphi}A_{1}^{\alpha}A_{23}^{\varphi}A_2^{\alpha}
...A_{(p-2)(p-1)}^{\varphi}B_{(p-1)p} + \newline A_{12}^{\varphi}A_{1}^{\alpha}A_{23}^{\varphi}A_{2}^{\alpha} ... A_{(p-1)p}^{\varphi}A_{(p-1)}^{\alpha}B_{p1}
$$

$$\cos(\theta/2) = (1 + a_{11} + a_{22} + a_{33})^{1/2} / 2$$
$$d\sin(\theta/2) = [b_1(a_{13} + a_{31}) + b_2(a_{23} + a_{32}) + b_3(1 - a_{11} - a_{22} + a_{33})] / \ 
 [2(1 - a_{11} - a_{22} + a{33})^{1/2}]$$
$$2\rho_1^2(1 - \cos(\theta)) + d^2 = b_1^2 + b_2^2 + b_3^2 = R$$

<img src="img/nucleotide_angles.png" alt="drawing" width="600"/>

In [2]:
mol = Molecule()
mol.read_pdb('example/example.pdb', model=4)
fig = to_plotly_figure(mol)
fig.update_layout(
    width=1200,
    height=800,
)
fig.show()

In [3]:
def get_theta(mol: Molecule):
    
    nucleotide_angles = [
        [("O3'", 1), "P", ("O5'", 2), ("C5'", 2)],
        ["P", ("O5'", 2), ("C5'", 2), ("C4'", 2)],
        [("O5'", 2), ("C5'", 2), ("C4'", 2), ("C3'", 2)],
        [("C5'", 1), ("C4'", 1), ("C3'", 1), ("O3'", 1)],
        [("C4'", 1), ("C3'", 1), ("O3'", 1), "P"],
        [("C3'", 1), ("O3'", 1), "P", ("O5'", 2)]
    ]

    torsion_angles_list = []
    for line in nucleotide_angles:
        res = []
        for atom in line:
            res.append(mol.get(*atom))
        torsion_angles_list.append(np.deg2rad(torsion_angle(*res)))

    nucleotide_angles_1d = [angle for row in nucleotide_angles for angle in row]
     
    result = []
    for i in range(1, len(nucleotide_angles_1d) - 1):
        chunk = nucleotide_angles_1d[i-1: i+2]
        if len(chunk) == 3:
            result.append(chunk)

    angles = [row[:3] for row in nucleotide_angles]

    bond_angles_list = []
    for line in angles:
        res = []
        for atom in line:
            res.append(mol.get(*atom))
        bond_angles_list.append(bond_angle(*res))

    result = np.eye(3)
    for i in range(6):
        result = result @ alpha_matrix(bond_angles_list[i]) @ phi_matrix(torsion_angles_list[i])
    
    return np.degrees(np.arccos(np.sqrt(1 + result.trace()) / 2) * 2)


def bond_angle(atom1, atom2, atom3):
    ba = atom1 - atom2
    bc = atom3 - atom2

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return angle

def phi_matrix(phi):
    mx = np.array([
        [1, 0, 0],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi), np.cos(phi)]
    ])
    return mx

def alpha_matrix(alpha):
    mx = np.array([
        [-np.cos(alpha), -np.sin(alpha), 0],
        [np.sin(alpha) , -np.cos(alpha), 0],
        [0, 0, 1]
    ])
    return mx

In [4]:
get_theta(mol)

178.28313581354732